In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Drop irrelevant features

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis= 1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Encoding 

In [4]:
# Encode categorical var

label_enc_gender = LabelEncoder()
data['Gender'] = label_enc_gender.fit_transform(data['Gender'])

In [7]:
data['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [16]:
# One hot encoding on geography
from sklearn.preprocessing import OneHotEncoder
ohe_geography = OneHotEncoder()
encoded_mat = ohe_geography.fit_transform(data[['Geography']])
encoded_mat

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [17]:
ohe_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [20]:
geo_encoded_df = pd.DataFrame(encoded_mat.toarray(), columns=ohe_geography.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [21]:
# combine encoded columns to original data
data = pd.concat([data.drop(['Geography'], axis =1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [22]:
# save the encoders and scaler 

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_enc_gender, file)


with open('OHE_geography.pkl', 'wb') as file:
    pickle.dump(ohe_geography, file)

In [23]:
# split the data in independent and dependent features

x = data.drop('Exited', axis = 1)
y = data['Exited']

In [41]:
# split data in train test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [42]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN Implementation

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [44]:
# Ann model building
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1], )),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                832       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# F/B propagration

model.compile(optimizer="adam", loss = 'binary_crossentropy', metrics=['accuracy'])

In [47]:
# setup tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = 'logs/fit'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir= log_dir, histogram_freq= 1)

In [48]:
# setup earlystopping
early_stopping_callback = EarlyStopping(monitor= 'val_loss', patience= 5, restore_best_weights=True)

In [49]:
# model traning
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs= 100, 
                    callbacks=[tensorflow_callback, early_stopping_callback])


Epoch 1/100
250/250 [==============================] - 4s 10ms/step - loss: 0.4615 - accuracy: 0.8000 - val_loss: 0.3946 - val_accuracy: 0.8280
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3885 - accuracy: 0.8400 - val_loss: 0.3594 - val_accuracy: 0.8545
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3584 - accuracy: 0.8539 - val_loss: 0.3495 - val_accuracy: 0.8560
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3464 - accuracy: 0.8589 - val_loss: 0.3441 - val_accuracy: 0.8560
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3402 - accuracy: 0.8615 - val_loss: 0.3446 - val_accuracy: 0.8565
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3377 - accuracy: 0.8619 - val_loss: 0.3418 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3335 - accuracy: 0.8620 - val_loss: 0.3457 - val_accuracy: 0.857

In [50]:
model.save('model.h5')

f:\LLMS\Deep Learning\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [51]:
# load tensor board extension
%load_ext tensorboard

In [53]:
%tensorboard --logdir logs/fit20250723-035436